In [3]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('hotel_final_preprocess.csv')
df

,City,Latitude,Longitude,Hotels,Places_to_visit,hotel_process,new_file_paths,hotel_final
0,Aamby Valley City,18.5726,73.4049,hotels/1.json,NaN,1_short.json,hotels/preprocessed_1_short.json,hotels/preprocessed_1_short.json
1,Abhaneri,27.0056,76.6102,hotels/2.json,NaN,2_short.json,hotels/preprocessed_2_short.json,hotels/preprocessed_2_short.json
2,Achooranam,11.5934,76.0131,hotels/3.json,NaN,3_short.json,hotels/preprocessed_3_short.json,hotels/preprocessed_3_short.json
3,Acharapakkam,12.4077,79.8170,hotels/4.json,NaN,4_short.json,hotels/preprocessed_4_short.json,hotels/preprocessed_4_short.json
4,Achrol,27.1330,75.9567,hotels/5.json,NaN,5_short.json,hotels/preprocessed_5_short.json,hotels/preprocessed_5_short.json
...,...,...,...,...,...,...,...,...
193,Bhavani,11.4471,77.6840,hotels/194.json,NaN,194_short.json,hotels/preprocessed_194_short.json,hotels/preprocessed_194_short.json
194,Bhavnagar,21.7645,72.1519,hotels/195.json,NaN,195_short.json,hotels/preprocessed_195_short.json,hotels/preprocessed_195_short.json
195,Bhawanipatna,19.9074,83.1642,hotels/196.json,NaN,196_short.json,hotels/preprocessed_196_short.json,hotels/preprocessed_196_short.json
196,Bhilai,21.1938,81.3509,hotels/197.json,NaN,197_short.json,hotels/preprocessed_197_short.json,hotels/preprocessed_197_short.json


In [29]:
import os
import json
from collections import Counter
from nltk.tokenize import word_tokenize

def process_reviews(json_file):
    good_words = []
    bad_words = []
    
    with open(json_file, 'r') as file:
        data = json.load(file)
        for place in data['places_with_reviews']:
            # Convert rating to float
            try:
                rating = float(place['rating'][0])
            except:
                rating = float("0")
            #print(type(rating))
            preprocessed_reviews = place['preprocessed_reviews']
        
            # Determine if review is good or bad based on rating
            is_good_review = True if rating > 3.5 else False
        
            # Tokenize and identify good and bad words
            for review in preprocessed_reviews:
                words = word_tokenize(review)
                if is_good_review:
                    good_words.extend(words)
                else:
                    bad_words.extend(words)
    
    return good_words, bad_words



# Dictionary to store aggregated good and bad words
aggregated_good_words = Counter()
aggregated_bad_words = Counter()

# Iterate through each file path
for file_path in df['hotel_final']:
    # Ensure the file exists
    if not isinstance(file_path, str) or not file_path:
        continue
    elif os.path.exists(file_path):
        #print(file_path)
        good_words, bad_words = process_reviews(file_path)
        aggregated_good_words.update(good_words)
        aggregated_bad_words.update(bad_words)
    else:
        print(f"File not found: {file_path}")

# Print aggregated good and bad words
print("Good words:")
print(aggregated_good_words.most_common(20))  # Print top 10 good words
print("\nBad words:")
print(aggregated_bad_words.most_common(20))   # Print top 10 bad words


Good words:
[('good', 4989), ('hotel', 3893), ('food', 3583), ('rooms', 3007), ('staff', 2782), ('stay', 2745), ('room', 2686), ('service', 2575), ('place', 2385), ('experience', 1766), ('also', 1679), ('clean', 1585), ('nice', 1479), ('great', 1314), ('location', 1227), ('one', 1195), ('well', 1162), ('best', 1146), ('resort', 1060), ('restaurant', 974)]

Bad words:
[('hotel', 408), ('room', 319), ('good', 317), ('rooms', 259), ('service', 166), ('stay', 153), ('place', 148), ('food', 137), ('staff', 123), ('also', 112), ('one', 110), ('dont', 109), ('water', 105), ('worst', 98), ('bad', 97), ('clean', 95), ('ac', 88), ('even', 73), ('location', 73), ('experience', 72)]


In [37]:
import pickle

def save_list_to_pickle(my_list, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(my_list, file)

save_list_to_pickle(aggregated_good_words, "good_words.pkl")
save_list_to_pickle(aggregated_bad_words, "bad_words.pkl")


User Interaction

In [4]:
#Read from pickle
import pickle

def read_list_from_pickle(file_path):
    with open(file_path, 'rb') as file:
        my_list = pickle.load(file)
    return my_list

good_words_counter = read_list_from_pickle("good_words.pkl")
good_words = list(good_words_counter.keys())
bad_words_counter = read_list_from_pickle("bad_words.pkl")
bad_words = list(bad_words_counter.keys())

print(good_words[:10])

['peaceful', 'retreat', 'experience', 'location', 'excellent', 'altogether', 'room', 'service', 'attractive', 'rooms']


In [27]:
# Determine the maximum length among good_words and bad_words
max_length = max(len(good_words), len(bad_words))

# Pad the shorter list with a placeholder word to match the maximum length
padding_word = "padding_word"

if len(good_words) < max_length:
    good_words += [padding_word] * (max_length - len(good_words))

if len(bad_words) < max_length:
    bad_words += [padding_word] * (max_length - len(bad_words))


In [32]:
import json
input = "Bengaluru" #take from user
df = pd.read_csv('hotel_final_preprocess.csv')
# Search for "Agra" in the "City" column
result = df[df['City'] == input]

# Check if there's any result
if not result.empty:
    # Get the filename from the "hotel/final" column
    filename = result.iloc[0]['hotel_final']

    # Load the JSON file
    with open(filename, 'r') as file:
        json_data = json.load(file)
        
    #print(json_data)  # Print or use the loaded JSON data
else:
    print("No entry found")

In [35]:
import json
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

places_with_reviews = json_data["places_with_reviews"]

all_words = good_words + bad_words
# Merge good_words and bad_words lists after converting them into sets to remove duplicates
all_words = list(set(good_words + bad_words))


# Function to remove special characters
def preprocess_review(review):
    # Remove non-ASCII characters
    review = re.sub(r'[^\x00-\x7F]+', ' ', review)
    # Remove punctuation and symbols, convert to lowercase
    review = re.sub(r'[^\w\s]', ' ', review).lower()
    # Remove duplicate words and stopwords
    stopwords = ['place']  # Add other stopwords as needed
    review = " ".join(word for word in set(review.split()) if word not in stopwords)
    return review


vectorizer = CountVectorizer(vocabulary=all_words, tokenizer=lambda text: text.split())

# Lists to store overall scores and place names
overall_scores = []
place_names = []
# List to store ranked places with original JSON objects
ranked_places_with_original = []
# Loop through each place with reviews
for place in places_with_reviews:
    place_name = place['name']
    print("Place:", place_name)
    
    # Extract reviews and preprocess them
    reviews = place['preprocessed_reviews']
    preprocessed_reviews = []
    problematic_reviews = []  # List to store problematic reviews
    for review in reviews:
        try:
            preprocessed_review = remove_special_characters(review.lower())
            preprocessed_review = " ".join(preprocessed_review.split())
            preprocessed_reviews.append(preprocessed_review)
        except Exception as e:
            problematic_reviews.append(review)
            print("Error processing review:", review)
            print("Error message:", str(e))
    
    # Transform reviews into vectors
    #print(preprocessed_reviews)
    #print("Vocabulary:", vectorizer.get_feature_names())
    review_vectors = vectorizer.fit_transform(preprocessed_reviews)

    # Calculate cosine similarity between reviews and good/bad words
    good_similarity = cosine_similarity(review_vectors, vectorizer.transform(good_words).toarray())
    bad_similarity = cosine_similarity(review_vectors, vectorizer.transform(bad_words).toarray())

    # Calculate average rating
    avg_rating = np.mean(place['rating'])

    # Calculate overall score
    overall_score = np.mean(good_similarity - bad_similarity) + avg_rating
    
    place_copy = place.copy()  # Make a copy to avoid modifying the original object
    place_copy['overall_score'] = overall_score
    ranked_places_with_original.append(place_copy)

# Rank the places based on overall scores (higher is better)
ranked_places_with_original = sorted(ranked_places_with_original, key=lambda x: x['overall_score'], reverse=True)

# Return the ranked places along with the original JSON objects
ranked_json_object = {"places_with_reviews": ranked_places_with_original}
print(json.dumps(ranked_json_object, indent=4))

Place: The LaLiT Ashok Bengaluru


/home/drishh207/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Place: The Leela Palace Bengaluru - Garden City's Only Modern Palace Hotel


/home/drishh207/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Place: ITC Gardenia, a Luxury Collection Hotel, Bengaluru


...

Place: Taj MG Road, Bengaluru


/home/drishh207/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Place: Conrad Bengaluru


/home/drishh207/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{
    "places_with_reviews": [
        {
            "name": "ITC Gardenia, a Luxury Collection Hotel, Bengaluru",
            "rating": [
                4.7
            ],
            "address": [
                "1, Residency Rd, Ashok Nagar, Bengaluru, Karnataka 560025, India"
            ],
            "reviews": [
                {
                    "text": "Very nice hotel that is a bit dated but amazing rooms. Their breakfast offering is really wonderful and the bar has such a nice vibe. The only negative is, they don't have a luggage room so don't be surprised if you see your checked bag out in the open at the lobby. I understand they have cameras and a lot of people around but it is just bad security practice since our bags not only have valuables but also important documents like passports. Find a small room out of sight and locked is my advice to management even if it is an ugly closet.",
                    "languageCode": "en",
                    "rating": [
          